In [266]:
from geopy.geocoders import Nominatim
from streamlit_folium import folium_static
import folium
import pandas as pd

In [267]:
locator = Nominatim(user_agent='myGeocoder')
location = locator.geocode('Dernburgstrasse 2')

In [268]:
print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))

Latitude = 52.5031532, Longitude = 13.2900053


In [269]:
#import data
df = pd.read_csv('data/denkmalliste_berlin_092020.csv')
df.shape

(12222, 12)

In [270]:
#drop not needed columns
df = df.drop(['Ensemble_Info', 'Zugehörigkeit', 'Adresse2', 'Architekt und weitere Informationen'], axis=1)

In [271]:
#remove null
df = df[df['Adresse'].notna()]
df.shape

(12003, 8)

In [272]:
#data cleaning and creating a new column
tmp_1 = df["Adresse"].str.split("/", n = 1, expand = True)
tmp_2 = tmp_1[0].str.split(",", n = 1, expand = True)
df['address'] = tmp_2[0]
df['address']

0                 Badenallee 29-30
1                    Badenallee 29
2              Binger Straße 10-16
3                Binger Straße 16 
4             Binger Straße 18-20 
                   ...            
12217          Schönblicker Straße
12218    Schönefelder Chaussee 100
12219            Späthstraße 77-81
12220       Wendenschloßstraße 254
12221         Schloßinsel Köpenick
Name: address, Length: 12003, dtype: object

In [273]:
df = df.sample(n=100)
df.shape

(100, 9)

In [275]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls (this min delay can be a problem)
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['address'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

2021-06-21 20:49:09.436 WARNING urllib3.connectionpool: Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=Torstra%C3%9Fe+62&format=json&limit=1
2021-06-21 20:49:31.659 WARNING urllib3.connectionpool: Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=Ahornallee+24&format=json&limit=1
2021-06-21 20:49:39.322 WARNING urllib3.connectionpool: Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=Drakestra%C3%9Fe+10&format=json&limit=1
202

In [276]:
df = df[df['latitude'].notna()]
df = df[df['longitude'].notna()]

In [281]:
df.shape

(99, 14)

In [282]:
#df.to_csv('monuments.csv')

In [283]:
m = folium.Map(location = [52.520008, 13.404954], tiles = "cartodbpositron", zoom_start=10 ) 

In [284]:
#m

In [285]:
df_monuments = df
for i,r in df_monuments.iterrows():
    location = [r["latitude"], r["longitude"]]
    folium.Marker(location=location, popup = r['ID'], tooltip=r['ID'], 
                 icon=folium.Icon( icon='eject')).add_to(m)

In [286]:
m